# Milestoning Calculations

Here we use milestoning to estimate free energy profiles and mean first passage times.

In [1]:
import matplotlib.pyplot as plt
import bkit.milestoning as milestoning
import numpy as np

%matplotlib ipympl

First we load the trajectory data. This is assumed to have already been projected onto some reasonably low-dimensional collective variable (CV) space $\mathbb{Y}$. In this particular case, $\mathbb{Y}\equiv$ (PC1-PC2 space) is of dimension 2.

In [2]:
base_dir = '/data/CDK8CycC-PL3/short_md/'
conf_dir_prefix = 'P'
ptraj_file = 'ptrajs.npz'

first_conf = 0
last_conf = 6000
step = 10

trajs = []
for conf in range(first_conf, last_conf+1, step):
    with np.load(f'{base_dir}{conf_dir_prefix}{conf}/{ptraj_file}') as data:
        trajs += [data[f] for f in data.files]
        
print(f'Loaded {len(trajs)} trajectories.')

Loaded 12020 trajectories.


We also need to keep track of the time resolution of the trajectory data (i.e., the observation interval $\Delta t$).

In [3]:
from pint import UnitRegistry
ureg = UnitRegistry()

dt = 0.1 * ureg.ns

In the previous notebook, we defined anchor points in $\mathbb{Y}$, which serve as centers of a Voronoi tessellation.

In [4]:
anchor_file = '/data/CDK8CycC-PL3/anchors.npy'
anchors = np.load(anchor_file)

Map each trajectory to a milestone <a href="https://ncatlab.org/nlab/show/schedule">schedule</a> $(a_1, t_1)\cdots(a_N, t_N)$. Here $a_1,\dots,a_N$ is the sequence of milestones visited, and $t_1,\dots,t_N$ are the corresponding lifetimes (in units of $\Delta t$).

In [5]:
schedules = milestoning.CoarseGraining(anchors, cutoff=1.).transform(trajs)

Estimate a Markovian milestoning model from the data. First we fit a `MarkovianMilestoningEstimator`.

In [6]:
estimator = milestoning.MarkovianMilestoningEstimator(dt=dt).fit(schedules)

We can fetch the maximum likelihood model by calling `fetch_model()`

In [7]:
model = estimator.maximum_likelihood_model

In [8]:
model.rate_matrix

Magnitude,[[-0.0662788532115117 0.06627885321151168 0.0 ... 0.0 0.0 0.0] [0.03628404555401823 -0.085150461743741 0.04886641618972274 ... 0.0 0.0 0.0] [0.0 0.04051801389461289 -0.08286680606276678 ... 0.0 0.0 0.0] ... [0.0 0.0 0.0 ... -0.06306007301692665 0.032359774311404695 0.0] [0.0 0.0 0.0 ... 0.026229034819612643 -0.05005173616959838 0.023822701349985726] [0.0 0.0 0.0 ... 0.0 0.05549560659781099 -0.055495606597811005]]
Units,1/nanosecond


In [ ]:
plt.figure()

kT = 0.593 # kcal/mol

plt.plot(range(model.n_states), model.free_energy(kT=kT))
plt.ylabel('Free energy (kcal/mol)')
_ = plt.xlabel('Milestone index')

In [9]:
source = 2
target = 40
print(f'MFPT({source} -> {target}) = {model.mfpt(target)[source] * 1e-10} ms')

MFPT(2 -> 40) = 0.0003351122826578615 ms


/home/jefft/miniconda3/lib/python3.7/site-packages/bkit/markov.py:120: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  mfpt[is_source] = np.linalg.solve(Q, -np.ones(len(Q)))


In [ ]:
posterior = estimator.sample_posterior(n_samples=100)
stats = posterior.gather_stats('mfpt', target_indices=target)
mean = stats.mean[source]
std = stats.std[source]
print(f'MFPT({source} -> {target}) = {mean * 1e-10} ms +/- {std * 1e-10} ms')

In [ ]:
stats = posterior.gather_stats('stationary_distribution')
stats.std

In [ ]:
Fs = [-RT * np.log(s.stationary_distribution) for s in posterior.samples]
F_mean = np.mean(Fs, axis=)
F_std = np.std([-RT * np.log(s.stationary_distribution) for s in posterior.samples], axis=0)

In [ ]:
target = 40
is_source = np.ones(model.n_states, dtype=bool)
is_source[target] = False
K = model.transition_kernel[is_source, :][:, is_source]
mfpt = np.zeros(self.n_states)
        mfpt[is_source] = np.linalg.solve(Q, -np.ones(len(Q)))
        return mfpt

In [ ]:
plt.figure()

n, bins, _ = plt.hist(np.concatenate(trajs), 1000, density=True, histtype='step')
plt.ylabel('Empirical Density')
_ = plt.xlabel('Reaction Coordinate ($\mathrm{\AA}$)')

In [ ]:
anchors = np.arange(14.0, 29.0, 1.0)[:, np.newaxis]

In [ ]:
model = MilestoningModel(anchors)
model.load_trajectory_data(trajs, dt=0.1)

In [ ]:
print(model.is_resolved)

In [ ]:
model.estimate()

In [ ]:
print(model.K)

In [ ]:
K = msmtools.estimation.transition_matrix(model._N, reversible=True)

In [ ]:
K[1,:]

In [ ]:
print('Mean lifetimes:')
for milestone, t, err in zip(model.milestones, model.t, model.t_stderr):
    print('{:8} : {:4.1f} ps +/- {:.1f} ps'.format(str(milestone), t, err))

In [ ]:
plt.close(plt.gcf())

fig, ax = plt.subplots(nrows=3, ncols=4, sharex=True, sharey=True)

for k in range(12):
    ax_ = ax[k // 4, k % 4]
    ax_.hist(model._lifetimes[k], bins=20, histtype='step', density=True)

    tmax = 40
    s = np.arange(0, tmax, 0.1)
    ax_.plot(s, np.exp(-s / model.t[k]) / model.t[k])

    ax_.set_xlim((0, tmax))
    
    ax_.text(tmax / 2, 0.16, model.milestones[k], horizontalalignment='center')
    
    if not k % 4:
        ax_.set_ylabel('P(t)')
    
    if k // 4 >= 2:
        ax_.set_xlabel('Lifetime (ps)')


In [ ]:
eigvals, eigvecs = scipy.linalg.eig(model.K, left=True, right=False)
q = np.real(eigvecs[:, np.isclose(eigvals, 1)].flatten())
if all(q < 0):
    q = -q
print(q)

In [ ]:
for i in range(len(K) - 1):
    j1 = q[i] * model.K[i, i+1]
    j2 = q[i+1] * model.K[i+1, i]
    print('{} - {} = {}'.format(j1, j2, j1 - j2))

In [ ]:
q[2] * model.K[2,1]

In [ ]:
eigvals, eigvecs = scipy.linalg.eig(K, left=True, right=False)
q = np.real(eigvecs[:, np.isclose(eigvals, 1)].flatten())
if all(q < 0):
    q = -q
print(q)

In [ ]:
p = q * model.t

In [ ]:
Q = np.zeros(K.shape)
for a in range(len(Q)):
    for b in range(len(Q)):
        if b == a:
            Q[a, a] = -1 / model.t[a]
        else:
            Q[a, b] = model.K[a, b] / model.t[a]

In [ ]:
plt.close(plt.gcf())
plt.figure()

RT = 0.593 # kcal/mol

err = RT * model.t_stderr / model.t
p = q * model.t
p = p / sum(p)
F = -RT * np.log(p)

plt.errorbar(range(len(q)), F, yerr=err)
plt.ylabel('Free energy (kcal/mol)')
_ = plt.xlabel('Milestone index')

In [ ]:
p_milestone = [0] + list(p) + [0]
p_cell = [(p_milestone[i] + p_milestone[i+1]) / 2 for i in range(len(p_milestone)-1)]

In [ ]:
plt.figure()

widths = [0.5] + [1.] * (len(p_cell) - 2) + [0.5]
p = np.divide(p_cell, widths)
plt.step(anchors.flatten(), -RT * np.log(p), where='mid')
plt.plot(anchors.flatten(), -RT * np.log(p), 'o--', color='grey', alpha=0.5)
plt.xlim([14, 28])
plt.ylabel('Free energy (kcal/mol)')
_ = plt.xlabel('Reaction coordinate ($\mathrm{\AA}$)')

In [ ]:
fig, ax = plt.subplots(2, 1, sharex=True)

widths = [0.5] + [1.] * (len(p_cell) - 2) + [0.5]
p = np.divide(p_cell, widths)
F = -RT * np.log(p)

x = [np.mean([bins[i-1], bins[i]]) for i in range(1, len(bins))]
F_emp = -RT * np.log(n)

F = F - min(F) + min(F_emp)

ax[0].step(anchors.flatten(), F, where='mid')
ax[0].plot(anchors.flatten(), F, 'o--', color='grey', alpha=0.5)
ax[0].set_xlim([14, 28])
ax[0].set_ylim([0, 4])
ax[0].set_ylabel('Milestoning')

ax[1].plot(x, F_emp)
ax[1].set_ylabel('Empirical')
ax[1].set_xlabel('Reaction coordinate ($\mathrm{\AA}$)')
ax[1].set_ylim([0, 4])

In [ ]:
reactant, product = 2, 13

# Apply cyclic boundary conditions
model.K[product, :] = 0
model.K[product, reactant] = 1

print(model.K)

In [ ]:
eigvals, eigvecs = scipy.linalg.eig(model.K, left=True, right=False)
q_cyc = np.real(eigvecs[:, np.isclose(eigvals, 1)].flatten())
if all(q < 0):
    q_cyc = -q_cyc
print(q_cyc)

In [ ]:
mfpt = np.dot(q_cyc, model.t) / q_cyc[product]
s = 'MFPT from milestone {} to milestone {}: {:.2f} ns'
print(s.format(model.milestones[reactant], model.milestones[product], mfpt / 1000))

Extract first hitting frames for a given milestone:

In [ ]:
milestone = {9, 10}

first_hitting_frames = {traj_id: [] for traj_id in traj_ids}

for traj_id, dtraj in zip(traj_ids, model._dtrajs):
    in_milestone = False
    for frame_index, (i, j) in enumerate(zip(dtraj[:-1], dtraj[1:])):
        if j == i:
            continue
        
        if {i, j} != milestone:
            in_milestone = False
            continue
            
        if not in_milestone:
            first_hitting_frames[traj_id].append(frame_index)
        in_milestone = True

In [ ]:
nframes = sum(map(len, first_hitting_frames.values()))
print(nframes)

In [ ]:
import pytraj as pt

In [ ]:
tn = '/data/p38a-SB2/complex.prmtop'
traj = pt.Trajectory(top=tn)
for traj_id, frame_indices in first_hitting_frames.items():
    if not frame_indices:
        continue
    fn = '/data/p38a-SB2/short_md/{}/c{}/r{}/mdcrd'.format(*traj_id)
    traj.append(pt.load(fn, tn, frame_indices=frame_indices))

In [ ]:
traj

In [ ]:
x = '%g' % (sum([anchors[i] for i in milestone]) / 2)
fn = '/home/jefft/milestone_{}.dcd'.format(x)
pt.save(fn, traj)
print('Saved first hitting frames as ' + fn)